# MLRun secret handling using Vault

This notebook demonstrates Vault secrets creation and handling in MLRun.


## Create a project & initialize Vault support
When a project is created, the `create_vault_secrets` command has to be used to request that the underlying framework is created that will enable Vault secrets to be used with this project. Calling this method on the project will create the following constructs for the project (if not already existing):

1. A k8s serviceaccount (`sa-vault-{project name}`)
2. A Vault policy (`mlrun-project-{project name}`) that enables access to secrets in the project path (`/secrets/secret/mlrun/projects/{proj name}`)
3. A Vault k8s role (`mlrun-role-project-{project name}`) that associates the SA's token with the policy

These configurations are performed on the MLRun API server side, not from the client.
In addition, the command passes a list of secret values to be associated with the project in Vault.

When a project was already initialized in Vault, those secrets can be examined using the `get_vault_secrets` method.
This method runs on the API server but used the client's Vault token, so client needs to be authorized to access Vault to execute it.

In [ ]:
import mlrun
import nuclio

In [ ]:
proj_name = 'vault-mlrun'

proj = mlrun.new_project(proj_name)

project_secrets = {'aws_key': '1234567890', 'github_key': 'proj1Key!!!'}
proj.create_vault_secrets(project_secrets)

proj.get_vault_secrets()

## Create and deploy a function utilizing Vault secrets

For this demo we'll use a simple function which gets a list of secret names to show, attempts to get their value from the context, and prints their contents.

In [ ]:
# nuclio: start-code

In [ ]:
def vault_func(context, secrets: list):
    """Validate that given secrets exists

    :param context: the MLRun context
    :param secrets: name of the secrets that we want to look at
    """
    context.logger.info("running function")
    for sec_name in secrets:
        sec_value = context.get_secret(sec_name)
        context.logger.info("Secret name: {}, value: {}".format(sec_name, sec_value))

    return True

In [ ]:
# nuclio: end-code

In [ ]:
func = mlrun.code_to_function(name='vault-func', 
                              kind='job',
                              image='mlrun/mlrun')

## Initialize function runtime
The `.with_secrets` function has a '`vault`' secret kind that will pass the specified Vault project secrets to the function context. The function spec
only contains the keys of the secrets ('aws_key' etc.) - the actual secret value is retrieved from Vault and planted in the function 
context in runtime.

In [ ]:
task = mlrun.NewTask(project=proj_name,
                     name='vault_test_run',
                     handler='vault_func',
                     params={'secrets':['github_key', 'aws_key']})

# Add access to project-level secrets
task.with_secrets('vault', ["aws_key"])

## Execute the function
Running the function using the task we've created will show only the value of the `aws_key` secret, since it's the only secret passed to the `task` object. Modifying the `with_secrets` command will result in different secrets being available to the function runtime.

In [ ]:
result = func.run(task)

By modifying the call to `with_secrets` so that all project secrets are passed to the runtime, the function will be able to present both the `github_key` and the `aws_key` secret values.

In [ ]:
# Access to all project-level secrets can be obtained by passing an empty list of secret names
task.with_secrets('vault', [])

result = func.run(task)

## Run the same function in another project's context
We will create a 2nd project, and assign different secret values to it. When the same function is executed in the new project's runtime context, it will get 
the new project's secrets. When running in this context, the function has no access to other projects' secrets.

In [ ]:
proj_name_2 = 'vault-mlrun-2'
proj2 = mlrun.new_project(proj_name_2)
proj2.create_vault_secrets({'aws_key': '0987654321', 'github_key': 'proj2Key???', 'password': 'myPassword'})

In [ ]:
task2 = mlrun.NewTask(project=proj_name_2,
                      name='vault_test_run_2',
                      handler='vault_func',
                      params={'secrets':['password', 'github_key', 'aws_key']})
task2.with_secrets('vault', ["aws_key", "github_key", "password"])

result = func.run(task2)

## Project-level client side Vault operations
Client-side Vault integration is possible, assuming that the pod hosting this notebook has Vault connectivity and is properly configured. 
Vault integration can be done using the `.with_secrets()` function, and then by using the `.get_secret()` utility function to extract secret value.

In [ ]:
proj.with_secrets('vault',['github_key'])
proj.get_secret('github_key')